## 이벤트 처리

In [ ]:
# p.52 키 이벤트
import cv2

img_file = './img/girl.jpg'
img = cv2.imread(img_file)
title = 'IMG'
x, y = 100, 100

while True:
    cv2.imshow(title, img)
    cv2.moveWindow(title, x, y)
    key = cv2.waitKey(0) & 0xFF
    print(key, chr(key))
    if key == ord('h'):
        x -= 10
    elif key == ord('j'):
        y += 10
    elif key == ord('k'):
        y -= 10
    elif key == ord('l'):
        x += 10
    elif key == ord('q') or key == 27: # 'q' 또는 'esc' 키이면 종료
        break
        cv2.destroyAllWindows()
    cv2.moveWindow(title, x, y)

In [ ]:
# p.53 mouse event
import cv2

title = 'mouse event'
img = cv2.imread('./img/blank_500.jpg')
cv2.imshow(title, img)

def onMouse(event, x, y, flags, param):
    print(event, x, y, )
    if event == cv2.EVENT_LBUTTONDOWN:
        cv2.circle(img, (x, y), 30, (0, 0, 0), -1)
        cv2.imshow(title, img)

cv2.setMouseCallback(title, onMouse)

while True:
    if cv2.waitKey(0) & 0xFF ==27:  # esc로 종료
        break
cv2.destroyAllWindows()

In [ ]:
# p.56 flag를 이용한 color circle
import cv2

title = 'mouse event'
img = cv2.imread('./img/blank_500.jpg')
cv2.imshow(title, img)

colors = {
    'black': (0, 0, 0),
    'red': (0, 0, 255),
    'blue': (255, 0, 0),
    'green': (0, 255, 0)
}

def onMouse(event, x, y, flags, param):
    print(event, x, y, flags)
    color = colors['black']
    if event == cv2.EVENT_LBUTTONDOWN:
        if flags & cv2.EVENT_FLAG_CTRLKEY and flags & cv2.EVENT_FLAG_SHIFTKEY:
            color = colors['green']
        elif flags & cv2.EVENT_FLAG_SHIFTKEY:
            color = colors['blue']
        elif flags & cv2.EVENT_FLAG_CTRLKEY:
            color = colors['red']

        cv2.circle(img, (x, y), 30, color, -1)
        cv2.imshow(title, img)
cv2.setMouseCallback(title, onMouse)

while True:
    if cv2.waitKey(0) & 0xFF == 27: # esc 버튼으로 종료
        break
    
cv2.destroyAllWindows()

In [ ]:
import cv2
import numpy as np

win_name = 'Trackbar'

img = cv2.imread('./img/blank_500.jpg')
cv2.imshow(win_name, img)

# 트랙바 이벤트 처리를 위한 함수
def onChange(x):
    print(x)
    r = cv2.getTrackbarPos('R', win_name)
    g = cv2.getTrackbarPos('G', win_name)
    b = cv2.getTrackbarPos('B', win_name)
    print(r, g, b)
    img[:] = [b, g, r]
    cv2.imshow(win_name, img)

cv2.createTrackbar('R', win_name, 255, 255, onChange)
cv2.createTrackbar('G', win_name, 255, 255, onChange)
cv2.createTrackbar('B', win_name, 255, 255, onChange)

while True:
    if cv2.waitKey(1) & 0xFF == 27:
        break

cv2.destroyAllWindows()

# 이미지 프로세싱 기초
## 관심 영역 (Region of Interest, ROI)
- 전체 이미지에서 연산과 분석의 대상이 되는 영역만을 지정하고 떼어내는 것을 관심영역을 지정한다고 함
- "roi = img[y:y+h, x:x+w]" (height, width 순, numpy에서는 row, column 순서라서...)

In [ ]:
import cv2
import numpy as np

img = cv2.imread('./img/sunset.jpg')

x = 320; y = 150; w = 50; h = 50
roi = img[y:y+h, x:x+w]

print(roi.shape)
cv2.rectangle(roi, (0, 0), (h-1, w-1), (0, 255, 0)) # roi에 사각형 그리기
cv2.imshow('img', img)

cv2.waitKey(0)
cv2.destroyAllWindows()

In [2]:
import cv2
import numpy as np

img = cv2.imread('./img/sunset.jpg')

x = 320; y = 150; w = 50; h = 50
roi = img[y:y+h, x:x+w]
img2 = roi.copy() # roi 배열 복제

img[y:y+h, x+w:x+w+w] = roi
cv2.rectangle(img, (x, y), (x+w+w, y+h), (0, 255, 0)) # roi에 사각형 그리기
cv2.imshow('img', img)

cv2.waitKey(0)
cv2.destroyAllWindows()

In [3]:
# p.109 마우스로 관심영역 지정
import cv2
import numpy as np

isDragging = False
x0, y0, w, h = -1, -1, -1, -1
blue, red = (255, 0, 0), (0, 0, 255)

def onMouse(event, x, y, flags, param):
    global isDragging, x0, y0, img
    if event == cv2.EVENT_LBUTTONDOWN:
        isDragging = True
        x0 = x
        y0 = y
    elif event == cv2.EVENT_MOUSEMOVE:
        if isDragging:
            img_draw = img.copy()
            cv2.rectangle(img_draw, (x0, y0), (x, y), blue, 2)
            cv2.imshow('img', img_draw)
    elif event == cv2.EVENT_LBUTTONUP:
        if isDragging:
            isDragging = False
            w = x - x0
            h = y - y0
            print('x: %d, y:%d, w:%d, h:%d' % (x0, y0, w, h))
            if w>0 and h>0:
                img_draw = img.copy()
                cv2.rectangle(img_draw, (x0, y0), (x, y), red, 2)
                cv2.imshow('img', img_draw)
                roi = img[y0:y0+h, x0:x0+w]
                cv2.imshow('cropped', roi)
                cv2.moveWindow('cropped', 0, 0)
                cv2.imwrite('./cropped.jpg', roi)
                print('cropped')
            else:
                cv2.imshow('img', img)
                print('좌측 상단에서 우측 하단으로 영역을 드래그 하시오')

img = cv2.imread('./img/sunset.jpg')
cv2.imshow('img', img)
cv2.setMouseCallback('img', onMouse)
cv2.waitKey()
cv2.destroyAllWindows()

x: 221, y:102, w:201, h:133
cropped


In [4]:
# p.112 selectROI
import cv2
import numpy as np

img = cv2.imread('./img/sunset.jpg')
x, y, w, h = cv2.selectROI('img', img, False)
if w and h:
    roi = img[y:y+h, x:x+w]
    cv2.imshow('cropped', roi)
    cv2.moveWindow('cropped', 0, 0)
    cv2.imwrite('./img/cropped2.jpg', roi)

cv2.imshow('img', img)
cv2.waitKey(0)
cv2.destroyAllWindows()